# Setup

In [1]:
# Get VM CPU and R version
shell_call <- function(command, ...) {
  result <- system(command, intern = TRUE, ...)
  cat(paste0(result, collapse = "\n"))
}
shell_call("grep -m1 'model name' /proc/cpuinfo | awk -F': ' '{printf \" CPU Model: %s \\n \",  $2}'")
shell_call("grep 'cpu cores' /proc/cpuinfo  | awk -F': ' '{a[cores]+=$2}END{printf \"CPU Cores: %s \\n \", a[cores] }'")
shell_call("grep MemTotal /proc/meminfo | awk '{printf \"RAM: %.1fGB \\n \", $2 / 1024 / 1024}'")
shell_call("R --version | head -n 1")

 CPU Model: Intel(R) Xeon(R) CPU @ 2.20GHz 
 CPU Cores: 72 
 RAM: 83.5GB 
 R version 4.4.1 (2024-06-14) -- "Race for Your Life"

In [2]:
# Get GPU Info
shell_call("nvidia-smi")

Tue Aug 27 02:19:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              47W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
# Install Torch with valid CUDA version

options(timeout = 600) # increasing timeout is recommended since we will be downloading a 2GB file.
# For Windows and Linux: "cpu", "cu117", "cu118" are the only currently supported
# For MacOS the supported are: "cpu-intel" or "cpu-m1"
kind <- "cu118"
version <- available.packages()["torch","Version"]
options(repos = c(
  torch = sprintf("https://torch-cdn.mlverse.org/packages/%s/%s/", kind, version),
  CRAN = "https://cloud.r-project.org" # or any other from which you want to install the other R dependencies.
))

install.packages("torch")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘coro’, ‘safetensors’




In [4]:
# Test Torch installation

library(torch)
torch_rand(4)

torch_tensor
 0.0936
 0.8286
 0.8919
 0.8154
[ CPUFloatType{4} ]

In [5]:
# Install BKTR

install.packages('BKTR')

### From Github (Latest Version)
# install.packages("devtools") # if not installed
# devtools::install_github("julien-hec/BKTR", ref = "main")

# For section 4 side by side plots
# install.packages('ggpubr')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘collections’, ‘png’, ‘plyr’, ‘jpeg’, ‘bitops’, ‘R6P’, ‘ggmap’




In [6]:
# The following block is because Jupyter for R does not print until the end of
# the code block execution. So, with this command we add real time printing.
# See: https://stackoverflow.com/questions/37689694/real-time-printing-to-console-with-r-in-jupyter
trace(what = "print", where = getNamespace("base"), exit = flush.console, print = FALSE)

Tracing function "print" in package "namespace:base"



[1] "print"

# Load Libraries

In [7]:
# Code to run BKTR examples #
# Lanthier, Lei, Sun and Labbe 2023 #

library('BKTR')
library(data.table)
library(ggplot2)
# library('ggpubr')

# Run BKTR

In [18]:

#######################
### 5.2: Imputation ###
#######################

df_res_arr <- c()
res_colnames <- c(
    'Lengthscale', 'Missing', 'Iter', 'B_MAE',
    'B_RMSE', 'Y_MAE', 'Y_RMSE', 'Time'
)

RANK_DECOMP <- 10
BURN_IN_ITER <- 500
SAMPLING_ITER <- 500

# Set seed and calculation params
TSR$set_params(seed = 1, fp_type = 'float32', fp_device = 'cuda')

# Run simulation for different lengthscale and missing percentage
for (len_scale in c(3, 6)) {
  for (miss_perc in c(0.1, 0.5, 0.9)) {
    for (i in 1:10) {
      # Adding some progression debug prints
      debug_msg <- sprintf(
        'Lengthscale: %d, Missing: %.1f, Iter: %02d',
        len_scale, miss_perc, i
      )
      print(debug_msg)

      spatial_kernel <- KernelMatern$new(
        smoothness_factor = 5,
        lengthscale = KernelParameter$new(value = len_scale)
      )
      temporal_kernel <- (
        KernelSE$new(lengthscale = KernelParameter$new(value = len_scale))
      )

      simu_data <- simulate_spatiotemporal_data(
        100, 150, 2, 10, 10,
        c(0, 2, 4), c(1, 3),
        spatial_kernel, temporal_kernel, 1
      )

      data_df <- simu_data$data_df
      index_choices_tsr <- TSR$tensor(1:nrow(data_df))
      nb_miss_index <- round(miss_perc * nrow(data_df))
      na_index <- as.numeric(
        TSR$rand_choice(index_choices_tsr, nb_miss_index)$cpu()
      )
      data_df$y[na_index] <- NA

      bktr_regressor <- BKTRRegressor$new(
        data_df = data_df,
        rank_decomp = RANK_DECOMP,
        burn_in_iter = BURN_IN_ITER,
        sampling_iter = SAMPLING_ITER,
        spatial_kernel = KernelMatern$new(smoothness_factor = 5),
        spatial_positions_df = simu_data$spatial_positions_df,
        temporal_kernel = KernelSE$new(),
        temporal_positions_df = simu_data$temporal_positions_df,
        has_geo_coords = FALSE
      )

      # Hide output of sampling because its volume is too large
      .unused_out <- capture.output(bktr_regressor$mcmc_sampling())

      # Calc Beta Errors
      y_err <- (
        bktr_regressor$imputed_y_estimates$y[na_index]
        - simu_data$data_df$y[na_index]
      )
      beta_err <- unlist(abs(
        lapply(bktr_regressor$beta_estimates[, -c(1, 2)], as.numeric)
        - simu_data$beta_df[, -c(1, 2)]
      ))
      y_rmse <- sqrt(mean(y_err^2))
      y_mae <- mean(abs(y_err))
      beta_rmse <- sqrt(mean(beta_err^2))
      beta_mae <- mean(abs(beta_err))

      # Formatting Values
      df_res_arr <- c(
        df_res_arr,
        len_scale,
        miss_perc,
        sprintf('%04d', i),
        sprintf('%.4f', beta_mae),
        sprintf('%.4f', beta_rmse),
        sprintf('%.4f', y_mae),
        sprintf('%.4f', y_rmse),
        sprintf('%.3f', as.numeric(
            bktr_regressor$result_logger$total_elapsed_time, units = "secs"
        ))
      )
    }
  }
}
df <- as.data.table(
    matrix(df_res_arr, ncol = length(res_colnames), byrow = TRUE)
)
colnames(df) <- res_colnames
print(df)

# Aggregate results (Table 5)
mean_fmt <- function(x) sprintf('%.4f', mean(x))
sd_fmt <- function(x) sprintf('%.4f', sd(x))

df <- df[, lapply(.SD, as.numeric), by = list(Lengthscale, Missing)]
df <- df[, .(
  B_MAE_avg = mean_fmt(B_MAE),
  B_MAE_sd = sd_fmt(B_MAE),
  B_RMSE_avg = mean_fmt(B_RMSE),
  B_RMSE_sd = sd_fmt(B_RMSE),
  Y_MAE_avg = mean_fmt(Y_MAE),
  Y_MAE_sd = sd_fmt(Y_MAE),
  Y_RMSE_avg = mean_fmt(Y_RMSE),
  Y_RMSE_sd = sd_fmt(Y_RMSE),
  Time_avg = mean_fmt(Time),
  Time_sd = sd_fmt(Time)
), by = list(Lengthscale, Missing)]
setkey(df, Lengthscale, Missing)
print(df)

[1] "Lengthscale: 3, Missing: 0.1, Iter: 01"
[1] "Lengthscale: 3, Missing: 0.1, Iter: 02"
[1] "Lengthscale: 3, Missing: 0.1, Iter: 03"
[1] "Lengthscale: 3, Missing: 0.1, Iter: 04"
[1] "Lengthscale: 3, Missing: 0.1, Iter: 05"
[1] "Lengthscale: 3, Missing: 0.1, Iter: 06"
[1] "Lengthscale: 3, Missing: 0.1, Iter: 07"
[1] "Lengthscale: 3, Missing: 0.1, Iter: 08"
[1] "Lengthscale: 3, Missing: 0.1, Iter: 09"
[1] "Lengthscale: 3, Missing: 0.1, Iter: 10"
[1] "Lengthscale: 3, Missing: 0.5, Iter: 01"
[1] "Lengthscale: 3, Missing: 0.5, Iter: 02"
[1] "Lengthscale: 3, Missing: 0.5, Iter: 03"
[1] "Lengthscale: 3, Missing: 0.5, Iter: 04"
[1] "Lengthscale: 3, Missing: 0.5, Iter: 05"
[1] "Lengthscale: 3, Missing: 0.5, Iter: 06"
[1] "Lengthscale: 3, Missing: 0.5, Iter: 07"
[1] "Lengthscale: 3, Missing: 0.5, Iter: 08"
[1] "Lengthscale: 3, Missing: 0.5, Iter: 09"
[1] "Lengthscale: 3, Missing: 0.5, Iter: 10"
[1] "Lengthscale: 3, Missing: 0.9, Iter: 01"
[1] "Lengthscale: 3, Missing: 0.9, Iter: 02"
[1] "Lengt

In [19]:

# Use 2 decimal places for the results
fmt_2dec <- function(x) {
  sprintf('%.2f', as.numeric(x))
}
res_df <- df[, lapply(.SD, fmt_2dec), by = list(Lengthscale, Missing)]

# Format in B_mae(avg±sd)/B_rmse(avg±sd) Y_mae(avg±sd)/Y_rmse(avg±sd)
res_df <- res_df[, .(
  B_res = paste(B_MAE_avg, '±', B_MAE_sd, '/', B_RMSE_avg, '±', B_RMSE_sd),
  Y_res = paste(Y_MAE_avg, '±', Y_MAE_sd, '/', Y_RMSE_avg, '±', Y_RMSE_sd),
  Time_res = paste(Time_avg, '±', Time_sd)
), by = list(Lengthscale, Missing)]
res_df

Lengthscale,Missing,B_res,Y_res,Time_res
<chr>,<chr>,<chr>,<chr>,<chr>
3,0.1,0.72 ± 0.18 / 1.14 ± 0.28,0.87 ± 0.02 / 1.10 ± 0.03,216.38 ± 5.20
3,0.5,0.63 ± 0.17 / 0.99 ± 0.32,0.90 ± 0.03 / 1.14 ± 0.05,211.65 ± 2.86
3,0.9,0.64 ± 0.11 / 0.93 ± 0.17,1.12 ± 0.05 / 1.45 ± 0.08,198.57 ± 3.08
6,0.1,0.22 ± 0.05 / 0.37 ± 0.10,0.81 ± 0.02 / 1.01 ± 0.02,200.82 ± 5.33
6,0.5,0.19 ± 0.05 / 0.30 ± 0.08,0.82 ± 0.01 / 1.03 ± 0.01,196.48 ± 3.93
6,0.9,0.27 ± 0.04 / 0.42 ± 0.06,0.89 ± 0.02 / 1.12 ± 0.02,181.49 ± 3.26
